## Analysis of Sequencing Data

**This is the analysis of the CBA whole cochlea (spiral ganglion + sensory epithelium) sequencing data.**<br>
This analysis includes quality control, clustering, and annotation. Previously done clustering and cell type annotation has been preserved and was used as a guide.

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scanpy as sc
import anndata as ad
import scipy.stats as st
import sys, os
import seaborn as sns
from matplotlib import rc_context

pd.set_option('display.max_columns', None)

sc.settings.set_figure_params(
    dpi=90,
    facecolor="white",
    color_map= "viridis_r")

### Reading Data

We read the data from an .h5ad file that contains the raw count matrix with some annotations and metadata.

Information carried over is:
- plate ID (a unique barcode for each 384-well plate sequenced)
- age of animals
- pre-annotated cell types (for UMAP and detailed projection)
- pre-annotated cell types corrected with batch integration
- UMAP embedding corrected with batch integration

In [4]:
# Reading the data
adata_original = sc.read_h5ad(r"C:\Users\Johann\Documents\Coding\Python\Master Thesis\Data\CBA_3mo_CompleteCochlea_Atlas_Annotated.h5ad")

# Making a clean copy ...
adata = ad.AnnData(adata_original.X.copy())

# ... and preserving some of the metadata as well as prior cell type annotations
adata.obs["plate"] = adata_original.obs["plateBC"].values
adata.obs["age"] = adata_original.obs["Age"].values
adata.obs["cell_types_pre"] = adata_original.obs["cell_types_for_UMAP"].values
adata.obs["cell_types_pre2"] = adata_original.obs["cell_types_for_detailed"].values
adata.obs["cell_types_pre_integrated"] = adata_original.obs["integrated_cell_type_updated"].values
adata.obsm["X_umap_integrated"] = adata_original.obsm["X_umap_integrated"]

adata

AnnData object with n_obs × n_vars = 4807 × 15095
    obs: 'plate', 'age', 'cell_types_pre', 'cell_types_pre2', 'cell_types_pre_integrated'
    obsm: 'X_umap_integrated'

### Quality Control

We calculate quality control metrics.

In [ ]:
# QC